In [2]:
import pandas as pd

In [12]:
df = pd.read_csv('combined_cleaned_file.csv')
df = df.drop(columns=['id'])
df.to_csv('combined_cleaned_file.txt', sep='\t', index=False)


In [17]:
import json
import os
import re # Import thư viện regular expression

def normalize_text(text):
    """
    Hàm làm sạch text:
    1. Thay thế các loại whitespace (newline, tab,...) bằng space thường.
    2. Gộp nhiều space liên tiếp thành một.
    3. Loại bỏ space thừa ở đầu/cuối.
    """
    if not isinstance(text, str):
        return "" # Trả về chuỗi rỗng nếu không phải string

    # Bước 1: Thay thế các loại whitespace bằng space thường
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    # Bước 2: Thay thế nhiều khoảng trắng liên tiếp bằng một khoảng trắng duy nhất
    # Dùng ' {2,}' để chỉ khớp với 2 hoặc nhiều dấu cách liền nhau
    text = re.sub(r' {2,}', ' ', text)

    # Bước 3: Loại bỏ khoảng trắng ở đầu và cuối chuỗi
    text = text.strip()
    return text

def clean_aspect_text(text):
    """
    Hàm làm sạch aspect_text: chuẩn hóa và loại bỏ dấu nháy kép bao quanh.
    Gọi normalize_text nhiều lần để đảm bảo tính nhất quán sau các bước xử lý.
    """
    if not isinstance(text, str):
        return ""
    # Bước 1: Chuẩn hóa text cơ bản lần đầu
    text = normalize_text(text)

    # Bước 2: Xử lý dấu nháy bao quanh (nếu có)
    cleaned_internally = False
    if text.startswith('"""') and text.endswith('"""'):
        text = text[3:-3].strip() # Bỏ dấu nháy và strip khoảng trắng lộ ra
        cleaned_internally = True
    elif text.startswith('"') and text.endswith('"'):
        text = text[1:-1].strip() # Bỏ dấu nháy và strip khoảng trắng lộ ra
        cleaned_internally = True

    # Bước 3: Nếu đã bỏ dấu nháy, chuẩn hóa lại bên trong lần nữa
    if cleaned_internally:
        text = normalize_text(text)

    # Bước 4: Trả về kết quả đã chuẩn hóa cuối cùng
    # (Gọi normalize_text lần cuối để đảm bảo chắc chắn, dù có thể thừa)
    return normalize_text(text)

# --- Hàm convert_to_semeval_json giữ nguyên phần còn lại ---
# (Copy lại toàn bộ hàm convert_to_semeval_json từ phiên bản V2
#  vì chỉ cần thay đổi 2 hàm helper normalize_text và clean_aspect_text)

def convert_to_semeval_json(input_filepath, output_filepath):
    """
    Chuyển đổi file text chứa dữ liệu ABSA sang định dạng JSON giống SemEval.
    (Phiên bản cải thiện xử lý khoảng trắng và dấu nháy - v3)

    Args:
        input_filepath (str): Đường dẫn đến file text đầu vào.
        output_filepath (str): Đường dẫn để lưu file JSON đầu ra.
    """
    sentences_data = {}
    found_count = 0
    not_found_count = 0
    found_in_sc_count = 0

    try:
        with open(input_filepath, 'r', encoding='utf-8') as f:
            header = next(f).strip().split('\t')
            print(f"Đã đọc header: {header}")

            for line_num, line in enumerate(f, 1):
                # Xử lý line đọc vào (giữ nguyên từ v2)
                line = line.replace('\n', ' ').replace('\r', '')
                parts = line.strip().split('\t')

                if len(parts) != 6:
                    print(f"Cảnh báo: Dòng {line_num} không có đủ 6 cột, bỏ qua: {line.strip()}")
                    continue

                review_orig, sentence_component_orig, aspect_text_orig, aspect_category, sentiment_text, sentiment = parts

                # --- Sử dụng hàm chuẩn hóa MỚI ---
                review = normalize_text(review_orig)
                cleaned_aspect_text = clean_aspect_text(aspect_text_orig)
                sentence_component = normalize_text(sentence_component_orig)
                # --- Hết phần sử dụng hàm chuẩn hóa MỚI ---

                if not cleaned_aspect_text:
                    # Kiểm tra aspect_text rỗng sau khi làm sạch (giữ nguyên từ v2)
                    if aspect_text_orig.strip(): # Chỉ cảnh báo nếu gốc không phải là rỗng/khoảng trắng
                         print(f"Cảnh báo: aspect_text ('{aspect_text_orig}') trở thành rỗng sau khi làm sạch ở dòng {line_num}. Bỏ qua.")
                    # Không cần in gì nếu gốc đã là khoảng trắng
                    continue


                sentiment = sentiment.lower()
                if sentiment not in ["positive", "negative", "neutral"]:
                     print(f"Cảnh báo: Sentiment không xác định '{sentiment}' ở dòng {line_num}, giữ nguyên.")


                start_index = -1
                end_index = -1
                term_to_store = aspect_text_orig # Mặc định lưu text gốc nếu không tìm thấy

                # Tìm vị trí 'from' và 'to' (logic giữ nguyên từ v2)
                start_index = review.find(cleaned_aspect_text)

                if start_index != -1:
                    end_index = start_index + len(cleaned_aspect_text)
                    term_to_store = cleaned_aspect_text
                    found_count += 1
                else:
                    # Không tìm thấy trong review, thử tìm trong sentence_component
                    # (In thông báo tìm kiếm) - Chỉ in nếu cleaned_aspect_text có nội dung
                    if cleaned_aspect_text:
                        print(f"Thông tin: Không tìm thấy '{cleaned_aspect_text}' trong review (dòng {line_num}). Thử tìm trong sentence_component.")

                    start_index_in_sc = sentence_component.find(cleaned_aspect_text)

                    if start_index_in_sc != -1:
                         if cleaned_aspect_text: # Chỉ in nếu tìm thấy text có nội dung
                              print(f"Thông tin: Tìm thấy '{cleaned_aspect_text}' trong sentence_component (dòng {line_num}).")
                         start_index_sc_in_review = review.find(sentence_component)

                         if start_index_sc_in_review != -1:
                             start_index = start_index_sc_in_review + start_index_in_sc
                             end_index = start_index + len(cleaned_aspect_text)
                             term_to_store = cleaned_aspect_text
                             found_in_sc_count += 1
                             print(f"Thông tin: Đã tính index tương đối với review cho dòng {line_num}.")
                         else:
                             if sentence_component: # Chỉ in lỗi nếu sentence_component không rỗng
                                 print(f"Lỗi: Tìm thấy aspect_text trong sentence_component, nhưng không tìm thấy sentence_component '{sentence_component[:50]}...' trong review ở dòng {line_num}. Gán from/to = -1.")
                             else: # SC rỗng thì không thể tìm thấy là đúng
                                  print(f"Thông tin: sentence_component rỗng ở dòng {line_num}, không thể tìm trong đó. Gán from/to = -1.")
                             not_found_count += 1
                             start_index = -1
                             end_index = -1
                    else:
                         # Không tìm thấy ở đâu cả
                         if cleaned_aspect_text: # Chỉ in lỗi nếu text tìm kiếm không rỗng
                             print(f"Lỗi: Không tìm thấy aspect_text '{cleaned_aspect_text}' (đã làm sạch) trong cả review và sentence_component ở dòng {line_num}. Gán from/to = -1.")
                         # Không cần in lỗi nếu cleaned_aspect_text rỗng
                         not_found_count += 1
                         start_index = -1
                         end_index = -1

                # Lưu thông tin aspect (logic giữ nguyên từ v2)
                aspect_info = {
                    "term": term_to_store,
                    "category": normalize_text(aspect_category),
                    "polarity": sentiment,
                    "from": str(start_index),
                    "to": str(end_index)
                }

                # Nhóm các aspect theo câu (logic giữ nguyên từ v2)
                review_key = review_orig.strip()
                if review_key not in sentences_data:
                    sentences_data[review_key] = {
                        "text": review_key,
                        "aspects": []
                    }

                if start_index != -1:
                     sentences_data[review_key]["aspects"].append(aspect_info)
                else:
                     if term_to_store.strip(): # Chỉ debug nếu term gốc không phải là khoảng trắng
                         print(f"Debug: Aspect không được thêm vào JSON (dòng {line_num}): {aspect_info}")


    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file đầu vào tại '{input_filepath}'")
        return
    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình đọc file: {e}")
        import traceback
        traceback.print_exc()
        return

    # Chuyển đổi và ghi file JSON (logic giữ nguyên từ v2)
    output_list = list(sentences_data.values())
    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump({"sentences": {"sentence": output_list}}, f, ensure_ascii=False, indent=4)
        print("-" * 20)
        print(f"Đã chuyển đổi thành công và lưu vào file: {output_filepath}")
        print(f"Thống kê tìm kiếm aspect_text:")
        print(f"- Tìm thấy trực tiếp trong Review: {found_count}")
        print(f"- Tìm thấy trong Sentence Component (tính được index): {found_in_sc_count}")
        print(f"- Không tìm thấy / Lỗi index / Bị rỗng: {not_found_count}") # Đổi tên cho rõ hơn
        total_processed = found_count + found_in_sc_count + not_found_count
        print(f"- Tổng số aspect đã xử lý (dòng): {total_processed}") # Tổng số dòng aspect đã đọc và xử lý
        # Tính tổng số aspect thực sự được thêm vào JSON cuối cùng
        final_aspect_count = sum(len(s['aspects']) for s in output_list)
        print(f"- Tổng số aspect được thêm vào JSON: {final_aspect_count}")
        print("-" * 20)

    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình ghi file JSON: {e}")


# --- Sử dụng hàm ---
input_file = 'combined_cleaned_file.txt'
# Đổi tên file output khác để không ghi đè lên file V2
output_file = 'output_semeval_format_v3.json'

if os.path.exists(input_file):
    convert_to_semeval_json(input_file, output_file)
else:
    print(f"Lỗi: File '{input_file}' không tồn tại trong thư mục hiện tại.")

Đã đọc header: ['Review', 'Sentence Component', 'aspect_text', 'aspect', 'sentiment_text', 'sentiment']
Thông tin: Không tìm thấy '\ngiảng viên dạy' trong review (dòng 16796). Thử tìm trong sentence_component.
Lỗi: Không tìm thấy aspect_text '\ngiảng viên dạy' (đã làm sạch) trong cả review và sentence_component ở dòng 16796. Gán from/to = -1.
Debug: Aspect không được thêm vào JSON (dòng 16796): {'term': '\\ngiảng viên dạy ', 'category': 'Teaching quality', 'polarity': 'negative', 'from': '-1', 'to': '-1'}
--------------------
Đã chuyển đổi thành công và lưu vào file: output_semeval_format_v3.json
Thống kê tìm kiếm aspect_text:
- Tìm thấy trực tiếp trong Review: 20777
- Tìm thấy trong Sentence Component (tính được index): 0
- Không tìm thấy / Lỗi index / Bị rỗng: 1
- Tổng số aspect đã xử lý (dòng): 20778
- Tổng số aspect được thêm vào JSON: 20777
--------------------


In [14]:
import json
import os

def format_to_simplified_visd4sa(input_filepath, output_filepath):
    """
    Chuyển đổi file text sang định dạng JSON Lines giống UIT-ViSD4SA,
    chỉ giữ lại text và danh sách các cặp [CATEGORY#POLARITY].

    Args:
        input_filepath (str): Đường dẫn đến file text đầu vào (tab-separated).
        output_filepath (str): Đường dẫn để lưu file JSON Lines đầu ra.
    """
    sentences_data = {} # Dictionary để nhóm các cặp Category#Polarity theo câu

    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            # Bỏ qua dòng header
            header = next(infile).strip().split('\t')
            print(f"Đã đọc header: {header}")

            for line_num, line in enumerate(infile, 1):
                parts = line.strip().split('\t')

                # Kiểm tra số lượng cột (cần ít nhất cột Review, aspect, sentiment)
                if len(parts) != 6:
                    print(f"Cảnh báo: Dòng {line_num} không có đủ 6 cột, bỏ qua: {line.strip()}")
                    continue

                # Lấy các cột cần thiết
                # Column 0: Review (text gốc)
                # Column 3: aspect (category)
                # Column 5: sentiment (polarity)
                review_text = parts[0].strip()
                category = parts[3].strip()
                polarity = parts[5].strip()

                # Chuẩn hóa polarity sang uppercase giống UIT-ViSD4SA
                # và xử lý các trường hợp có thể có
                if polarity.lower() == 'positive':
                    polarity_formatted = 'POSITIVE'
                elif polarity.lower() == 'negative':
                    polarity_formatted = 'NEGATIVE'
                elif polarity.lower() == 'neutral':
                    polarity_formatted = 'NEUTRAL'
                else:
                    print(f"Cảnh báo: Polarity không xác định '{polarity}' ở dòng {line_num}. Sử dụng giá trị gốc.")
                    # Quyết định cách xử lý: bỏ qua, dùng giá trị gốc, hoặc gán mặc định
                    # Ở đây ta dùng giá trị gốc và viết hoa
                    polarity_formatted = polarity.upper()

                # Tạo chuỗi "CATEGORY#POLARITY"
                label_string = f"{category}#{polarity_formatted}"

                # Thêm vào dictionary, sử dụng set để tránh trùng lặp nhãn cho cùng một câu
                if review_text not in sentences_data:
                    sentences_data[review_text] = set() # Dùng set để tự động loại bỏ trùng lặp
                sentences_data[review_text].add(label_string)

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file đầu vào tại '{input_filepath}'")
        return
    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình đọc file input: {e}")
        return

    # Ghi ra file JSON Lines
    try:
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            count = 0
            for text, labels_set in sentences_data.items():
                # Tạo đối tượng dictionary cho dòng JSON
                json_object = {
                    "text": text,
                    "labels": sorted(list(labels_set)) # Chuyển set thành list, sắp xếp để nhất quán (tùy chọn)
                }
                # Chuyển dictionary thành chuỗi JSON
                json_line = json.dumps(json_object, ensure_ascii=False)
                # Ghi chuỗi JSON vào file, theo sau là ký tự xuống dòng
                outfile.write(json_line + '\n')
                count += 1
        print(f"Đã chuyển đổi thành công và lưu {count} dòng vào file: {output_filepath}")

    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình ghi file JSON Lines: {e}")

# --- Sử dụng hàm ---
input_file = 'combined_cleaned_file.txt' # File input của bạn
output_file = 'output_simplified_visd4sa_format.jsonl' # File output định dạng JSON Lines (.jsonl)

# Kiểm tra xem file input có tồn tại không
if os.path.exists(input_file):
    format_to_simplified_visd4sa(input_file, output_file)
else:
    print(f"Lỗi: File '{input_file}' không tồn tại trong thư mục hiện tại.")

Đã đọc header: ['Review', 'Sentence Component', 'aspect_text', 'aspect', 'sentiment_text', 'sentiment']
Đã chuyển đổi thành công và lưu 15519 dòng vào file: output_simplified_visd4sa_format.jsonl


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ file CSV
df = pd.read_csv('combined_cleaned_file.csv')

# Ánh xạ aspect -> số nguyên
aspect_mapping = {
    "Course information": 0,
    "General review": 1,
    "Learning environment": 2,
    "Organization and management": 3,
    "Support from lecturers": 4,
    "Teaching quality": 5,
    "Test and evaluation": 6,
    "Workload": 7
}

# Ánh xạ sentiment -> số nguyên
sentiment_mapping = {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
}

# Thay thế aspect và sentiment bằng số tương ứng
df["aspect_encoded"] = df["aspect"].map(aspect_mapping)
df["sentiment_encoded"] = df["sentiment"].map(sentiment_mapping)

df = df.drop(columns=['id'])
df = df.rename({'Sentence Component': "text"})
# Chia lần 1: train (70%) và temp (30%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Chia lần 2: temp thành test (20%) và val (10%)
test_df, val_df = train_test_split(temp_df, test_size=1/3, random_state=42)  # 1/3 của 30% là 10%

# Kiểm tra kích thước
print(f'Train: {len(train_df)}')
print(f'Test: {len(test_df)}')
print(f'Validation: {len(val_df)}')

# Lưu lại nếu cần
train_df.to_csv('uit_train_absa.csv', index=False)
test_df.to_csv('uit_test_absa.csv', index=False)
val_df.to_csv('uit_val_absa.csv', index=False)


Train: 14545
Test: 4156
Validation: 2078


In [18]:
import json
import os
from collections import defaultdict # Sử dụng defaultdict để nhóm dễ dàng hơn

def format_for_asca_json(input_filepath, output_filepath):
    """
    Chuyển đổi file text sang định dạng JSON cho tác vụ ASCA.

    Args:
        input_filepath (str): Đường dẫn đến file text đầu vào (tab-separated).
        output_filepath (str): Đường dẫn để lưu file JSON đầu ra.
    """
    # Sử dụng defaultdict(set) để tự động xử lý câu mới và tránh trùng lặp cặp (category, polarity)
    sentences_data = defaultdict(set)

    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            header = next(infile).strip().split('\t')
            print(f"Đã đọc header: {header}")

            for line_num, line in enumerate(infile, 1):
                # Loại bỏ ký tự xuống dòng trong chính dòng đọc vào
                line = line.replace('\n', ' ').replace('\r', '')
                parts = line.strip().split('\t')

                if len(parts) != 6:
                    print(f"Cảnh báo: Dòng {line_num} không có đủ 6 cột, bỏ qua: {line.strip()}")
                    continue

                # Lấy các cột cần thiết
                review_text = parts[0].strip()
                category = parts[3].strip()
                polarity = parts[5].strip().lower() # Lấy polarity và chuẩn hóa lowercase

                # Kiểm tra giá trị polarity chuẩn
                if polarity not in ["positive", "negative", "neutral"]:
                    print(f"Cảnh báo: Polarity không chuẩn '{polarity}' ở dòng {line_num}. Sử dụng giá trị gốc (viết thường).")
                    # Bạn có thể quyết định xử lý khác ở đây nếu muốn
                    # polarity = "neutral" # Ví dụ: Gán mặc định là neutral

                # Thêm cặp (category, polarity) vào set của câu tương ứng
                # Dùng tuple vì set yêu cầu phần tử hashable
                if review_text and category: # Chỉ thêm nếu review và category không rỗng
                    sentences_data[review_text].add((category, polarity))

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file đầu vào tại '{input_filepath}'")
        return
    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình đọc file input: {e}")
        import traceback
        traceback.print_exc()
        return

    # Chuyển đổi dữ liệu đã nhóm thành định dạng list JSON mong muốn
    output_list = []
    for text, label_tuples in sentences_data.items():
        # Chuyển set các tuple thành list các dictionary
        aspect_categories_list = [
            {"category": cat, "polarity": pol}
            for cat, pol in sorted(list(label_tuples)) # Sắp xếp để thứ tự nhất quán (tùy chọn)
        ]

        # Tạo đối tượng sentence
        sentence_object = {
            "text": text,
            "aspectCategories": aspect_categories_list
        }
        output_list.append(sentence_object)

    # Tạo cấu trúc JSON cuối cùng (ví dụ: đặt trong key "sentences")
    final_json_structure = {"sentences": output_list}

    # Ghi ra file JSON
    try:
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            json.dump(final_json_structure, outfile, ensure_ascii=False, indent=4)
        print("-" * 20)
        print(f"Đã chuyển đổi thành công sang định dạng ASCA và lưu vào file: {output_filepath}")
        print(f"Tổng số câu được xử lý: {len(output_list)}")
        print("-" * 20)

    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình ghi file JSON: {e}")

# --- Sử dụng hàm ---
input_file = 'combined_cleaned_file.txt' # File input của bạn
output_file = 'output_asca_format.json' # File output định dạng JSON cho ASCA

if os.path.exists(input_file):
    format_for_asca_json(input_file, output_file)
else:
    print(f"Lỗi: File '{input_file}' không tồn tại trong thư mục hiện tại.")

Đã đọc header: ['Review', 'Sentence Component', 'aspect_text', 'aspect', 'sentiment_text', 'sentiment']
--------------------
Đã chuyển đổi thành công sang định dạng ASCA và lưu vào file: output_asca_format.json
Tổng số câu được xử lý: 15519
--------------------
